# AI app builder (Week 4)

- This project allows LLMs to build applications using the create file tools, llms can specify a path as well as code content allowing them build simple applicatiions

- projects built with the llms are stored in the projects folder

- potential improvements i'll like to work on includes adding the search functionality, allowing the LLM search and view content of files, an edit functionality, a delete functionality and possible a functionality to run commands on the host machine

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import os
import json

load_dotenv()



True

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")


In [38]:
openai_client = OpenAI(api_key=openai_api_key)
openrouter_client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=openrouter_api_key)
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

#setup models and clients
models = ["gpt-4o-mini", "gpt-5", "gpt-oss"]
clients = {
    "gpt-4o-mini": openai_client,
    "gpt-5": openai_client,
    "gpt-oss": ollama_client
}

In [39]:
#setup system prompt
system_prompt = f"""
you are an expert in building applications using the tools provided
every app must have a parent folder set in the path
use proper industry standard practices and document
make the interface modern user friendly and responsive
use unsplash for images

when you build an app, you should always follow these steps:
1. Define the app's purpose and functionality
2. Design the app's interface
3. Implement the app's functionality using the tools provided
4. Test the app
5. Refactor the app to improve its readability and efficiency
6. Write documentation for the app
7. Final response should include how the user can run the app as well as summary of all actions you performed and why

"""

In [40]:
# function to create a file and write to it, could receive a file name, content, and a path

def create_file(file_name, content, path):
    static_path = "./projects/"
    # Strip leading slashes so path is always relative to projects/ (avoids absolute path like /cleaning-company-website)
    path = path.lstrip("/")
    full_dir = os.path.join(static_path, path)

    # Create directory if it doesn't exist
    os.makedirs(full_dir, exist_ok=True)

    # Create file
    full_path = os.path.join(full_dir, file_name)
    with open(full_path, "w") as file:
        file.write(content)

    return f"File {file_name} created successfully at {full_dir}"
#create function json schema for model
create_file_function = {
    "name": "create_file",
    "description": "Create a file and write to it, could receive a file name, content, and a path, path can be the folder you want to create the file in or folder path",
    "parameters": {
        "type": "object",
        "properties": {
            "file_name": {"type": "string"},
            "content": {"type": "string"},
            "path": {"type": "string"}
        },
        "required": ["file_name", "content", "path"]
    }
}



In [41]:
tools = [{"type": "function", "function": create_file_function}]
toolsdict = {"create_file": create_file}
# function to call tool

def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        print(tool_name, tool_args)
        tool_to_call = toolsdict[tool_name]
        if tool_to_call is None:
            response = "Tool doesn't exist",
            
        else:
            print("tool_name", tool_name)
            print("tool_args", tool_args)
            response = tool_to_call(**tool_args)

        # Tool response content must be a string for the API
        content = response if isinstance(response, str) else json.dumps(response)
        responses.append({
            "role": "tool",
            "content": response,
            "tool_call_id": tool_call.id
        })
    return responses


In [42]:
# chat function to interact with the model, receives a message and returns a response
def chat(model, prompt):
    client = clients[model]
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
    )

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = client.chat.completions.create(model=model, messages=messages, tools=tools)


    return response.choices[0].message.content



In [ ]:
with gr.Blocks() as app:
    gr.Markdown("## AI App Builder")

    with gr.Row():
        with gr.Column():
            description = gr.TextArea( label="What would you like to build?", placeholder="What would you like to build?")
            model = gr.Dropdown(choices=models, value=models[0], label="Model")
        with gr.Column():
            output = gr.Markdown(label="Output")
    with gr.Row():
       submit = gr.Button("Generate App")

    submit.click(fn=chat, inputs=[model, description], outputs=[output])

app.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


create_file {'file_name': 'app.py', 'content': 'import os\nfrom datetime import datetime\nfrom typing import Any, Dict\n\nfrom flask import Flask, jsonify, request\nfrom flask_cors import CORS\nfrom dotenv import load_dotenv\n\nfrom utils import get_price_history, ValidationError\nfrom model import train_and_forecast\n\n\nload_dotenv()\n\ndef create_app() -> Flask:\n    app = Flask(__name__)\n\n    # Configure CORS for local dev; in production, tighten allowed origins\n    CORS(app, resources={r"/api/*": {"origins": os.getenv("ALLOWED_ORIGINS", "*")}})\n\n    @app.get("/api/health")\n    def health() -> Any:\n        return jsonify({"status": "ok", "time": datetime.utcnow().isoformat() + "Z"})\n\n    @app.post("/api/predict")\n    def predict() -> Any:\n        try:\n            payload: Dict[str, Any] = request.get_json(force=True, silent=False) or {}\n            ticker: str = (payload.get("ticker") or "").upper().strip()\n            start: str | None = payload.get("start_date")\n  